# Advanced methods for driving your network with synapic spike-trains

## Example: Using The Poisson Spike Generator

## Example: Using spikes from CSV

## Example: Realistic stimuli with FilterNet

## Example: Incorporating real data with NWB

While you can use tools like FilterNet or PoissonSpikeGenerator to create theoretical-to-realistic synaptic stimuli onto a network from a variety of different modes. But better yet, when actual experimental data is available, it is often possible to just use that. Especially as as more-and-more experimental electrophysiological data sets are being made publically available through resources like [DANDI](https://dandiarchive.org/) or [The Allen Brain Observatory](https://portal.brain-map.org/circuits-behavior/visual-coding-neuropixels) we will want to not only use such data as a base-line for validating and comparing our models, but to actually use the data within specific simulations. For example, when modeling a network of one population and/or region, we will want use recordings of surrounding cells to help excite and inhibit our cells in a more realistic manner.

Traditionally a major issue with using encorporating experimental ephys data into simulations is trying to parse the wide variety of different ways the data was stored. Luckily, the [Neurodata Without Borders (NWB)](https://www.nwb.org/) has developed a field-adopted format for storing experimental data. BMTK can take these files and automatically insert them into simulations.

In this example we will take the previous model of the Mouse Primary Visual Cortex and add inputs that we know come from higher cortical regions (in this case just the VisL and Hippocampal regions) along with the input from the LGN. For these added regions, we will use actual neuropixels recordings of activity from the region during drifting gratings

### Step 1: Download the data

First step is to find experimental NWB that includes spiking events. For our example we will use data from Allen Institute Visual Coding Data downloaded using the AllenSDK. We will get a three experimental sessions that we know contains recordings the the VisL and hippocamus.

In [5]:
from allensdk.brain_observatory.ecephys.ecephys_project_cache import EcephysProjectCache

cache = EcephysProjectCache.from_warehouse(
    manifest='./ecephys_cache_dir/neuropixels.manifest.json'
)
cache.get_session_data(715093703)
cache.get_session_data(798911424)
cache.get_session_data(754829445)

Downloading: 100%|████████████████████████████████████████████████████████| 2.86G/2.86G [00:25<00:00, 111MB/s]
/local1/.local/miniconda-23.10.0/envs/bmtk-py3.10/lib/python3.10/site-packages/hdmf/utils.py:668: UserWarning: Ignoring cached namespace 'hdmf-common' version 1.1.3 because version 1.8.0 is already loaded.
  return func(args[0], **pargs)
/local1/.local/miniconda-23.10.0/envs/bmtk-py3.10/lib/python3.10/site-packages/hdmf/utils.py:668: UserWarning: Ignoring cached namespace 'core' version 2.2.2 because version 2.6.0-alpha is already loaded.
  return func(args[0], **pargs)
Downloading: 100%|████████████████████████████████████████████████████████| 2.86G/2.86G [00:25<00:00, 113MB/s]
/local1/.local/miniconda-23.10.0/envs/bmtk-py3.10/lib/python3.10/site-packages/hdmf/utils.py:668: UserWarning: Ignoring cached namespace 'hdmf-common' version 1.1.3 because version 1.8.0 is already loaded.
  return func(args[0], **pargs)
/local1/.local/miniconda-23.10.0/envs/bmtk-py3.10/lib/python3.10/

By defaul the NWB files will be downloaded into the *ecephys_cache_dir*. Since nwb files are essentially just structured HDF5 files you can use tools like [HDFView](https://www.hdfgroup.org/downloads/hdfview/) or [h5py](https://www.h5py.org/) to read them once they have been downloaded. 

### Step 2: Connecting VISL and Hippocampal onto our V1 cells.

To simulation synaptic stimulation from the VisL and Hippocampal cell recordings onto our V1 model, we will need to create a population of virtual cells representing the new cells and synapses. Unforantely the electrophyiology data doesn't include information about network geometry so it is up to the modeler to decide how to connection the experimental data into our network.

As we did before we will separate node populations called 'VISl' and 'Hippocampus' and use the NetworkBuilder to create feedforward synaptic connections

```python
visl = NetworkBuilder('VISl')
visl.add_nodes(
    N=n_visl_units,
    model_type='virtual',
    ...
)
visl.add_edges(
    source=visal.nodes(),
    target=visp.nodes(ei='e'), 
    connection_rule=connection_rule_e2e,
    dynamics_params='AMPA_ExcToExc.json',
    model_template='Exp2Syn',
    ...
)

```

See the *./build_network.nwb_inputs.py* for the full script that builds the SONATA network found in *./network_nwb_inputs/*

### Step 3: Updating the configuration file to include NWB data.

Before we can run the simulation we must update the SONATA configuration file so that the simulation:
1. Knows which .nwb files to fetch spiking data from
2. Knows how to map cells (eg. NWB units) from our experimental data to cells (eg. SONATA nodes) in our 'VISl' and 'hippocampus' populations.
3. Know which interval of the experimental data to use in our simulation.

The most straight forward way of doing this is to add the following to our configuration file (*config.nwb_inputs.json*) in the "inputs" section:

```json
"inputs": {
    "hippo_spikes": {
        "input_type": "spikes",
        "module": "ecephys_probe",
        "node_set": "hippocampus",
        "input_file": "./ecephys_cache_dir/session_715093703/session_715093703.nwb",
        "units": {
            "location": ["CA1", "CA3", "Po"]
        }
        "mapping": "sample",
        "interval": {
            "interval_name": "drifting_gratings",
            "temporal_frequency": 4.0,
            "orientation": 90
        }

    },
    "visl_spikes": {
        "input_type": "spikes",
        "module": "ecephys_probe",
        "node_set": "VISl",
        "input_file": "./ecephys_cache_dir/session_715093703/session_715093703.nwb",
        "mapping": "sample",
        "units": {
            "location": "VISl",
        }
        "interval": {
            "interval_name": "drifting_gratings",
            "temporal_frequency": 4.0,
            "orientation": 90
        },
    }
}
```
* The **input_type** and **module** will always be set to values `spikes` and `ecephys_probe`, respectively, when importing extracellular electrophysiology NWB files into your simulation.
* The **node_set** is the subset of cells in our network to use as virtual cells that are generating spikes.
* The **input_file** in the name of the nwb file(s) to use for spikes. To use data from multiple sessions just use a list of files:
```json
    "input_file": [
        "./ecephys_cache_dir/session_715093703/session_715093703.nwb",
        "./ecephys_cache_dir/session_798911424/session_798911424.nwb",
        "./ecephys_cache_dir/session_754829445/session_754829445.nwb"
    ]
```
* The **units** field tell us which units from the nwb file to take their spiking data from based on either specifc keywords and/or unit-id. In the Neuropixels NWB files each unit has an field called "location" to determine which region the data came from, which we can use here to tell that for our model's "hippocampus" cells use any data coming from either the CA1, CA3 or Po regions.
* **mapping** tells how to map NWB **units** -> SONATA **node_set**. Setting the value to `sample` to do a random mapping without replacement. You can also use options `sample_with_replacment`, useful if you have more nodes in your model than units in your data. Or if you have a specific mapping from NWB unit_ids to SONATA node_ids you can use optionn `units_map` (in which case **units** will point to a csv file).
* **interval** tells simulation which interval of time to fetch spikes from. Inside the NWB file there is a stimulus table that marks the stimuli at any given epoch of time. We use this table to only get spikes recorded which a "drifting grating" stimuli was present with a given orientation and frequency. If you know the specific time you can also use option
```json
    "interval": [start_time_ms, end_time_ms]
```

And finally we are ready to run our simulation

In [7]:
from bmtk.simulator import bionet

bionet.reset()
conf = bionet.Config.from_json('config.nwb_inputs.json')
conf.build_env()

net = bionet.BioNetwork.from_config(conf)
sim = bionet.BioSimulator.from_config(conf, network=net)
sim.run()

2024-05-03 15:07:39,747 [INFO] Created log file


INFO:NEURONIOUtils:Created log file


2024-05-03 15:07:39,870 [INFO] Building cells.


INFO:NEURONIOUtils:Building cells.


2024-05-03 15:07:49,864 [INFO] Building recurrent connections


...

INFO:NEURONIOUtils:    step:15000 t_sim:1500.00 ms


2024-05-03 15:12:04,649 [INFO]     step:20000 t_sim:2000.00 ms


INFO:NEURONIOUtils:    step:20000 t_sim:2000.00 ms


2024-05-03 15:12:04,700 [INFO] Simulation completed in 4.0 minutes, 1.87 seconds 


INFO:NEURONIOUtils:Simulation completed in 4.0 minutes, 1.87 seconds 


In [ ]:
## Example: Dynamically generate custom spikes inputs

## Example: Forcing spotonaneous synaptic activity within a network.

So far in this tutorials we've focused on generating stimuli using synaptic stimuli coming from outside our main modeled network. In other tutorials we should different types in input to drive a simulation including current-clamps, voltage-clamps, and extracellar stimulation. While these can be generate both the kinds of stimuli we might see in the experiments and living brains, they tend to not be very grainular, especially when we want to study the secondary effects of activity within a network.

One option that BMTK gives use for having more grainular control of internal network activity is by forcing certain synapses to spontaneously fire at pre-determined times. Not only can this give us more control of network dynamics that would be much harder to achive using current clamps or feedforward spike trains. But it also let's us isolate external activity from recurrent activity.

In BMTK this is done by adding a new input type to the "inputs" section the SONATA config with **input_type** and **module** called `syn_activity`. And the minimum we must define the pre-synaptic cells that will spontaneously fire and a list of firing times:

```json
"syn_activity": {
  "input_type": "syn_activity",
  "module": "syn_activity",
  "precell_filter": {
      "population": "VISp",
      "ei": "e"
  },
  "timestamps": [500.0, 1000.0, 1500.0, 2000.0, 2500.0, 3000.0, 3500.0]
}
```
* **precell_filter** determines the synapses to sponataneously activate based on the presynaptic/source cell. In this case we tell BMTK sponataneous activity to apply on to synapses with a source-cell that has attributes `population==VISp` and `ei==e`. If you know exactly which cells you want to use you can filter by `node_id`:
```json
    "node_id": [0, 1, 2, 3],
```
* **timestamps** is a list of timestamps, in milliseconds, to activate the neuron following startup. If you have too many timestamps to add to the json directly, you can also pass in a string path to a txt file where each line is a timestamp.


In the above example, all the synapses with VISp, exc pre-synaptic connections will fire at the given timestamp. For further grainular control you can all set the **postcell_filter** too for filtering out synapses based on post-synaptic cell. For example if you want spontaneous firing in exc -> inh connections (the above example would also include exc -> exc synapses:

```json
"syn_activity": {
  "input_type": "syn_activity",
  "module": "syn_activity",
  "precell_filter": {
      "population": "VISp",
      "ei": "e"
  },
  "postcell_filter": {
      "population": "VISp",
      "ei": "i"
  },
  "timestamps": [500.0, 1000.0, 1500.0, 2000.0, 2500.0, 3000.0, 3500.0]
}
```



In [ ]:
from bmtk.simulator import bionet

bionet.reset()
conf = bionet.Config.from_json('config.spont_syns.json')
conf.build_env()

net = bionet.BioNetwork.from_config(conf)
sim = bionet.BioSimulator.from_config(conf, network=net)
sim.run()